# 07: Procesamiento de Votaciones

**Propósito:** Este *notebook* consolida todos los archivos `detalle.csv` (crudos, por período) de `data/01_raw/` en un único archivo maestro de votaciones.

**Estrategia (Manejo de Memoria):**
Debido a que el *dataset* final de votaciones tendrá millones de filas, no podemos cargar todos los CSV a la vez.

1.  **Iterar:** Se itera por cada período.
2.  **Cargar Chunk:** Se carga un solo `detalle.csv` en memoria.
3.  **Procesar:** Se llama a `process_votaciones_chunk` (de `src/processing_utils.py`) para limpiar, mapear y estandarizar ese *chunk*.
4.  **Anexar (Append):** El *chunk* limpio se anexa al archivo `votaciones_master_clean.parquet` en `data/02_processed/`.
5.  **Liberar y Repetir.**

**Dependencias:**
* `data/01_raw/periodos_master.csv`
* `data/01_raw/[periodo]/detalle.csv` (Múltiples archivos)

**Salidas (Artifacts):**
* `data/02_processed/votaciones_master_clean.parquet` (Un único archivo grande)

In [6]:
import pandas as pd
from pathlib import Path
import sys
import logging
import os # <-- Para eliminar el archivo antiguo
from tqdm.notebook import tqdm
import pyarrow as pa          # <-- AÑADIR ESTA LÍNEA
import pyarrow.parquet as pq

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import process_votaciones_chunk
    from src.common_utils import sanitize_filename
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

In [7]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

# Dependencia
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"

# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "votaciones_master_clean.parquet"

logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

2025-10-29 12:51:23,563 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\votaciones_master_clean.parquet


In [8]:
if OUTPUT_FILE.exists():
    logging.warning(f"Eliminando archivo Parquet existente: {OUTPUT_FILE}")
    try:
        os.remove(OUTPUT_FILE)
        logging.info("Archivo antiguo eliminado.")
    except Exception as e:
        logging.error(f"No se pudo eliminar el archivo. Verifique los permisos. Error: {e}")
        raise
else:
    logging.info("No existe archivo Parquet anterior. Se creará uno nuevo.")

2025-10-29 12:51:23,926 - INFO - No existe archivo Parquet anterior. Se creará uno nuevo.


## 2. Carga de Dependencias (Períodos)

Cargamos la lista maestra de períodos para saber qué carpetas iterar.

In [9]:
try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
except FileNotFoundError:
    logging.error(f"ERROR FATAL: No se encontró {MASTER_PERIOD_FILE}")
    raise

2025-10-29 12:51:24,688 - INFO - Se cargó la lista maestra de 10 períodos.


## 3. Bucle Principal de Procesamiento (Chunking)

Iteramos sobre cada período, cargamos su `detalle.csv`, lo procesamos en memoria y lo anexamos al archivo Parquet en el disco.

In [11]:
logging.info(f"Iniciando procesamiento de votaciones para {len(df_periodos)} períodos...")
total_rows_processed = 0

# --- 1. Inicializar el 'Writer' de Parquet ---
writer = None
global_schema = None

try:
    for row in tqdm(df_periodos.itertuples(), total=len(df_periodos), desc="Procesando Períodos"):
        
        periodo_nombre = row.Nombre
        carpeta_periodo = DATA_DIR_RAW / sanitize_filename(periodo_nombre)
        ruta_detalle = carpeta_periodo / "detalle.csv"
        
        if not ruta_detalle.exists():
            logging.warning(f"No se encontró {ruta_detalle}, saltando período {periodo_nombre}.")
            continue
            
        logging.info(f"--- Procesando Chunk: {periodo_nombre} ---")
        
        # 1. Cargar Chunk
        df_raw = pd.read_csv(ruta_detalle, low_memory=False)
        
        # 2. Procesar Chunk 
        df_clean = process_votaciones_chunk(df_raw, periodo_nombre)
        
        if df_clean.empty:
            logging.warning(f"Chunk limpio está vacío para {periodo_nombre}. No se anexa nada.")
            continue
            
        # 3. Convertir el chunk de Pandas a una Tabla de PyArrow
        table = pa.Table.from_pandas(df_clean)
        
        # --- 4. Lógica de Escritura (Append) ---
        if writer is None:
            # 4a. Guardar el schema y abrir el 'writer'
            global_schema = table.schema
            writer = pq.ParquetWriter(OUTPUT_FILE, global_schema)
            
            writer.write_table(table)
        else:
            # 4b. Escribir el chunk
            writer.write_table(table.cast(global_schema))

        total_rows_processed += len(df_clean)
        logging.info(f"Anexadas {len(df_clean)} filas de {periodo_nombre}.")

except Exception as e:
    # Capturar el período problemático en el mensaje de error
    logging.error(f"ERROR FATAL al procesar período {periodo_nombre}: {e}", exc_info=True)
finally:
    # --- 5. (CRÍTICO) Cerrar el 'Writer' ---
    if writer:
        writer.close()
        logging.info("--- Procesamiento de votaciones finalizado ---")
        logging.info(f"Escritor Parquet cerrado. Total de filas procesadas: {total_rows_processed}")

2025-10-29 12:52:26,185 - INFO - Iniciando procesamiento de votaciones para 10 períodos...


Procesando Períodos:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-29 12:52:26,199 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1965-1969\detalle.csv, saltando período 1965-1969.
2025-10-29 12:52:26,200 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1990-1994\detalle.csv, saltando período 1990-1994.
2025-10-29 12:52:26,202 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1994-1998\detalle.csv, saltando período 1994-1998.
2025-10-29 12:52:26,203 - WARNING - No se encontró C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\1998-2002\detalle.csv, saltando período 1998-2002.
2025-10-29 12:52:26,203 - INFO - --- Procesando Chunk: 2002-2006 ---
2025-10-29 12:52:27,227 - INFO - Filtrando 'Proyectos de Ley' para 2002